In [38]:
import numpy as np


In [39]:
d = np.array([[1, 2, 6, 4, 10, 8], 
              [4, 1, 3, 5, 9, 7],
              [5, 2, 3, 5, 4, 8]])


gt = np.array([[1, 1, 1, 1, 1, 1], 
              [2, 2, 2, 2, 2, 2],
              [3, 3, 3, 3, 3, 3]])


print(d.shape)

print(d)

(3, 6)
[[ 1  2  6  4 10  8]
 [ 4  1  3  5  9  7]
 [ 5  2  3  5  4  8]]


In [41]:
mu_1, mu_2, mu_3 = 2, 5, 10
beta = 10


mu = np.array([mu_1, mu_2, mu_3])


S = np.zeros_like(d, dtype=int)

# For each pixel, calculate distance to each mean and assign to closest
for i in range(d.shape[0]):
    for j in range(d.shape[1]):
        # Calculate squared distances to each class mean
        distances = [(d[i,j] - mean)**2 for mean in mu]
        # Assign pixel to class with minimum distance
        # Note: Class labels are 0, 1, 2, etc. (not 1, 2, 3)
        S[i,j] = np.argmin(distances)


print(S)

[[0 0 1 1 2 2]
 [1 0 0 1 2 1]
 [1 0 0 1 1 2]]


In [42]:
def segmentation_energy(S, D, mu, beta):
    # Step 1: Compute intensity realization
    intensity_realization = np.zeros_like(D)
    for i in range(len(mu)):
        intensity_realization[S == i] = mu[i]
    
    # Step 2: Calculate likelihood energy (V1)
    V1 = np.sum((D - intensity_realization)**2)
    
    # Step 3: Calculate prior energy (V2)
    V2 = 0
    
    # Horizontal neighbors
    horizontal_diff = (S[:, :-1] != S[:, 1:])
    V2 += np.sum(horizontal_diff) * beta
    
    # Vertical neighbors
    vertical_diff = (S[:-1, :] != S[1:, :])
    V2 += np.sum(vertical_diff) * beta
    
    return V1, V2

In [43]:
V1, V2 = segmentation_energy(S, gt, mu, beta)

In [45]:
print(V1)

350
